## PyConsult

#### PiConsult functionality in a desktop form using Tkinter

In [1]:
# Import packages
import tkinter as tk
import tkinter.ttk as ttk

# from ctypes import windll # This allows us to change the DPI settings for the application

import numpy as np
import serial
import serial.tools.list_ports
import time
from Main_Bare_Imports.Settings import Load_Config, Save_Config

from Main_Bare_Imports.Connection import PortConnect, ECU_Connect
from Main_Bare_Imports.Read import ReadStream

In [2]:
# Load data from config
CONF = 'configJSON.json' # config file
Config = Load_Config(CONF)

In [3]:
PORT,BYPASSED, ECU_CONNECTED = None, False, False

In [4]:
def launch_data_stream():
    global landing, PORT, ECU_CONNECTED, BYPASSED
    # landing.destroy() # close original window
    
    if ECU_CONNECTED == False:
        PopUp = tk.Tk()
        PopUp.title("Error")
        PopUp.geometry("300x100")
        tk.Label(PopUp, text="Please connect to the ECU first").pack()
        tk.Button(PopUp, text="OK", command=PopUp.destroy).pack()

    else:
        DataStreamWindow()

def launch_dtc():
    global landing, PORT, ECU_CONNECTED, BYPASSED
    # landing.destroy() # close original window
    
    if ECU_CONNECTED == False:
        PopUp = tk.Tk()
        PopUp.title("Error")
        PopUp.geometry("300x100")
        tk.Label(PopUp, text="Please connect to the ECU first").pack()
        tk.Button(PopUp, text="OK", command=PopUp.destroy).pack()

    else:
        DataStreamWindow()

def launch_testing():
    global landing, PORT, ECU_CONNECTED, BYPASSED
    # landing.destroy() # close original window
    
    if ECU_CONNECTED == False:
        PopUp = tk.Tk()
        PopUp.title("Error")
        PopUp.geometry("300x100")
        tk.Label(PopUp, text="Please connect to the ECU first").pack()
        tk.Button(PopUp, text="OK", command=PopUp.destroy).pack()

    else:
        DataStreamWindow()
    
# def launch_settings():
#     global landing
#     landing.destroy() # close original window
#     SettingsWindow()



In [5]:
def Connection():
    global PORT,BYPASSED, ECU_CONNECTED
    # This function is triggered by pressing connect on landing screen
    start_time = time.time() # Start timer for connection
    while PORT is None and BYPASSED == False:
        PORT,BYPASSED = PortConnect(PORT,BYPASSED) # Connect to the port
        if time.time() - start_time > 3: # If it takes longer than 3 seconds, break
            break
    if PORT is not None:
        ECU_CONNECTED,BYPASSED = ECU_Connect(PORT,ECU_CONNECTED,BYPASSED) # Connect to the ECU 

    # Create a popup window to show connection status
    popup = tk.Tk()
    popup.title("Connection Status")
    popup.geometry("300x100")
    if ECU_CONNECTED == True:
        label = tk.Label(popup, text="Connected to ECU")
    else:
        label = tk.Label(popup, text="Failed to connect to ECU")
    label.pack(pady=20)
    button = tk.Button(popup, text="OK", command=popup.destroy)
    button.pack(pady=10)
    popup.mainloop()



In [6]:
var = 0
def inc():
    global var
    var += 1
    return var

In [9]:
import threading

### Data Stream Window ###
def DataStreamWindow():
    global PORT,BYPASSED, ECU_CONNECTED, Config
    R = ReadStream(port=PORT, daemon=True)
    # Create a new window
    DataStream = tk.Tk()
    DataStream.title("Data Stream")
    DataStream.geometry("800x600")
    DataStream.resizable(False, False)

    # Create labels for the names and the data with a box around them
    frame_width = 200
    frame_height = 100


    RPMframe = tk.Frame(DataStream, relief=tk.RIDGE, borderwidth=2, width=frame_width, height=frame_height)
    RPMframe.grid_propagate(False)  # Prevent frame from resizing to fit contents
    RPMlabel = tk.Label(RPMframe, text="RPM", font=("Arial", 20))
    RPMvalue = tk.Label(RPMframe, text=var, font=("Arial", 20))

    Speedframe = tk.Frame(DataStream, relief=tk.RIDGE, borderwidth=2, width=frame_width, height=frame_height)
    Speedframe.grid_propagate(False)
    Speedlabel = tk.Label(Speedframe, text="Speed", font=("Arial", 20))
    Speedvalue = tk.Label(Speedframe, text=str(R.SPEED_Value), font=("Arial", 20))

    TPSframe = tk.Frame(DataStream, relief=tk.RIDGE, borderwidth=2, width=frame_width, height=frame_height)
    TPSframe.grid_propagate(False)
    TPSlabel = tk.Label(TPSframe, text="TPS", font=("Arial", 20))
    TPSvalue = tk.Label(TPSframe, text=str(R.TPS_Value), font=("Arial", 20))

    # Place the labels in the window
    RPMlabel.pack(side=tk.LEFT, padx=10, pady=10)
    RPMvalue.pack(side=tk.RIGHT, padx=10, pady=10)
    RPMframe.grid(row=0, column=0, padx=10, pady=10)

    Speedlabel.pack(side=tk.LEFT, padx=10, pady=10)
    Speedvalue.pack(side=tk.RIGHT, padx=10, pady=10)
    Speedframe.grid(row=0, column=1, padx=10, pady=10)

    TPSlabel.pack(side=tk.LEFT, padx=10, pady=10)
    TPSvalue.pack(side=tk.RIGHT, padx=10, pady=10)
    TPSframe.grid(row=0, column=2, padx=10, pady=10)

    i = 0
    def update_rpm():
        while True:
            time.sleep(0.1)
            inc()
            RPMvalue.config(text=var)
            DataStream.update_idletasks()

    # Start the RPM update in a separate thread
    rpm_thread = threading.Thread(target=update_rpm, daemon=True)
    rpm_thread.start()

    DataStream.mainloop()


In [10]:
DataStreamWindow()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "c:\Users\kylec\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\kylec\Dropbox\vscode\PyConsult\Main_Bare_Imports\Read.py", line 79, in run
    self.port.write(bytes([0x5A,0x0B,0x5A,0x01,0x5A,0x08,0x5A,0x0C,0x5A,0x0D,0x5A,0x03,0x5A,0x05,0x5A,0x09,0x5A,0x13,0x5A,0x16,0x5A,0x17,0x5A,0x1A,0x5A,0x1C,0x5A,0x21,0xF0]))
    ^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'write'
Exception in thread Thread-6 (update_rpm):
Traceback (most recent call last):
  File "c:\Users\kylec\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\kylec\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\kylec\AppData\Local\Temp\ipykernel_13888\2746487237.py", line 51, in update_rpm
  File "c:\Users\kylec\anaconda3\Lib\tkinter\__init__.py", line 1702, in configure
    return self._con

: 

In [52]:
### Settings Window ###
def SettingsWindow():
    Config = Load_Config(CONF)
    # GUI parameters
    padx = 5
    pady = 5
    geox = 460
    geoy = 300

    # Create settings window
    settings = tk.Tk()
    settings.title("PyConsult Settings")
    settings.geometry(str(geox) + "x" + str(geoy))
    settings.resizable(True,True)
    settings.focus_force()
    # windll.shcore.SetProcessDpiAwareness(1) # Uncomment to enable DPI awareness, Windows only, needs items resized.

    

    # Create input boxes for settings
    rpm_warning = ttk.Entry(settings, text="RPM Warning")
    rpm_warning.insert(0,Config["RPM_Warning"])

    temp_warning = ttk.Entry(settings, text="Temperature Warning")
    temp_warning.insert(0,Config["Coolant_Warning"])

    speed_units = ttk.Combobox(settings, values=["MPH","KPH"])
    speed_units.set(Config["Units_Speed"])

    temp_units = ttk.Combobox(settings, values=["F","C"])
    temp_units.set(Config["Units_Temp"])

    stock_tire_width = ttk.Entry(settings, text="Stock Tire Width")
    stock_tire_width.insert(0,Config["Stock_Tire_Width"])
    stock_tire_ar = ttk.Entry(settings, text="Stock Aspect Ratio")
    stock_tire_ar.insert(0,Config["Stock_Tire_AR"])
    stock_tire_diam = ttk.Entry(settings, text="Stock Tire Diameter")
    stock_tire_diam.insert(0,Config["Stock_Tire_Diam"])

    new_tire_width = ttk.Entry(settings, text="New Tire Width")
    new_tire_width.insert(0,Config["New_Tire_Width"])
    new_tire_ar = ttk.Entry(settings, text="New Aspect Ratio")
    new_tire_ar.insert(0,Config["New_Tire_AR"])
    new_tire_diam = ttk.Entry(settings, text="New Tire Diameter")
    new_tire_diam.insert(0,Config["New_Tire_Diam"])

    stock_diff = ttk.Entry(settings, text="Stock Final Drive")
    stock_diff.insert(0,Config["Stock_Final"])
    new_diff = ttk.Entry(settings, text="New Final Drive")
    new_diff.insert(0,Config["New_Final"])

    # Create labels for input boxes
    rpm_warning_label = ttk.Label(settings, text="RPM Warning")
    temp_warning_label = ttk.Label(settings, text="Temperature Warning")

    speed_units_label = ttk.Label(settings, text="Speed Units")
    temp_units_label = ttk.Label(settings, text="Temp Units")

    stock_label = ttk.Label(settings, text="Stock")
    stock_label.grid(row=12,column=1)
    new_label = ttk.Label(settings, text="New")
    new_label.grid(row=12,column=2)

    ar_label = ttk.Label(settings,text="AR")
    width_label = ttk.Label(settings,text="Width")
    diam_label = ttk.Label(settings,text="Diameter")
    diff_label = ttk.Label(settings,text="Final Drive")

    # Place input boxes and labels in grid
    #Warnings
    rpm_warning.grid(row=0,column=1, padx=padx, pady=pady)
    rpm_warning_label.grid(row=0,column=0, padx=padx, pady=pady)

    temp_warning.grid(row=1,column=1, padx=padx, pady=pady)
    temp_warning_label.grid(row=1,column=0, padx=padx, pady=pady)

    #Units
    speed_units.grid(row=2,column=1, padx=padx, pady=pady)
    speed_units_label.grid(row=2,column=0, padx=padx, pady=pady)

    temp_units.grid(row=3,column=1, padx=padx, pady=pady)
    temp_units_label.grid(row=3,column=0, padx=padx, pady=pady)

    # Speed correction
    #Place labels
    width_label.grid(row=13,column=0)
    ar_label.grid(row=14,column=0)
    diam_label.grid(row=15,column=0)

    stock_tire_width.grid(row=13,column=1, padx=padx, pady=pady)
    stock_tire_ar.grid(row=14,column=1, padx=padx, pady=pady)
    stock_tire_diam.grid(row=15,column=1, padx=padx, pady=pady)

    new_tire_width.grid(row=13,column=2, padx=padx, pady=pady)
    new_tire_ar.grid(row=14,column=2, padx=padx, pady=pady)
    new_tire_diam.grid(row=15,column=2, padx=padx, pady=pady)

    # differential
    stock_diff.grid(row=16,column=1, padx=padx, pady=pady)
    new_diff.grid(row=16,column=2, padx=padx, pady=pady)
    diff_label.grid(row=16,column=0, padx=padx, pady=pady)

    # Save button
    def Save(CONF,Config):
        Config["RPM_Warning"] = rpm_warning.get()
        Config["Coolant_Warning"] = temp_warning.get()
        Config["Units_Speed"] = speed_units.get()
        Config["Units_Temp"] = temp_units.get()
        Config["Stock_Tire_Width"] = stock_tire_width.get()
        Config["Stock_Tire_AR"] = stock_tire_ar.get()
        Config["Stock_Tire_Diam"] = stock_tire_diam.get()
        Config["New_Tire_Width"] = new_tire_width.get()
        Config["New_Tire_AR"] = new_tire_ar.get()
        Config["New_Tire_Diam"] = new_tire_diam.get()
        Config["Stock_Final"] = stock_diff.get()
        Config["New_Final"] = new_diff.get()
        Save_Config(CONF, Config)
        
        # Create a popup window to indicate that settings have been saved
        popup = tk.Toplevel()
        popup.title("Save Confirmation")
        popup.geometry("200x100")
        label = ttk.Label(popup, text="Settings saved successfully!")
        label.pack(pady=20)
        ok_button = ttk.Button(popup, text="OK", command=popup.destroy)
        ok_button.pack()


    save_button = ttk.Button(settings, text="Save", command=lambda: Save(CONF, Config))
    save_button.grid(row=17,column=1)

    settings.mainloop()


In [53]:
### Main landing/startup window ###
# GUI parameters
def LandingWindow():
    global landing
    ipadx = 40
    ipady = 40
    padx = 10
    pady = 10
    geox = 460
    geoy = 360#280
    buttonwidth = 20

    # Create main window
    landing = tk.Tk()
    landing.title("PyConsult GUI")
    landing.geometry(str(geox) + "x" + str(geoy))
    landing.resizable(False,False)
    landing.focus_force()

    # Create buttons and labels
    selectionlabel = ttk.Label(landing, text="Select Mode!",width=10,font=("Arial", 16))
    ds_button = ttk.Button(landing, text="Data Stream",width=buttonwidth,command=launch_data_stream)
    dtc_button = ttk.Button(landing, text="DTCs",width=buttonwidth,command=launch_dtc)
    test_button = ttk.Button(landing, text="Testing",width=buttonwidth,command=launch_testing)
    setting_button = ttk.Button(landing, text="Settings",width=buttonwidth,command=SettingsWindow)
    connect_button = ttk.Button(landing, text="Connect",width=buttonwidth,command=Connection)
    coms_box = ttk.Combobox(landing, values=serial.tools.list_ports.comports())

    ds_button.grid(row=1,column=0, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 
    dtc_button.grid(row=2,column=0, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 
    test_button.grid(row=1,column=1, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 
    setting_button.grid(row=2,column=1, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 
    connect_button.grid(row=3,column=1, ipadx=ipadx/4, ipady=ipady/4,padx=padx, pady=pady)
    coms_box.grid(row=3,column=0, ipadx=ipadx/4, ipady=ipady/4,padx=padx, pady=pady)

    selectionlabel.grid(row=0,column=0,columnspan=2)

    landing.mainloop()


In [54]:
LandingWindow()